In [3]:
import sqlite3
conn = sqlite3.connect('morphology.db')
c = conn.cursor()

In [2]:
c.execute("""
SELECT DISTINCT nouncase FROM lex
""")
c.fetchall()

[('nominatīvs',),
 ('ģenitīvs',),
 ('datīvs',),
 ('akuzatīvs',),
 ('lokatīvs',),
 ('vokatīvs',),
 ('',)]

In [22]:
c.execute("DROP VIEW IF EXISTS splitlex")
c.execute("""
CREATE VIEW splitlex AS
SELECT DISTINCT
    LOWER(SUBSTR(word, LENGTH(word)-x+1, x)) as char,
    x as num,
    word,
    nouncase
FROM lex
INNER JOIN (
    WITH RECURSIVE
      cnt(x) AS (
         SELECT 1
         UNION ALL
         SELECT x+1 FROM cnt
         LIMIT 8
      )
    SELECT x FROM cnt
) AS numbers
ON numbers.x <= LENGTH(word)
""")
c.fetchall()

[]

In [29]:
c.execute("DROP TABLE IF EXISTS nouncase")
c.execute("""
CREATE TABLE nouncase AS
SELECT * FROM (
    SELECT
        splitlex.char, splitlex.num, splitlex.nouncase,
        chartotal.total AS chartotal, COUNT(DISTINCT splitlex.word) AS cnt,
        COUNT(DISTINCT splitlex.word) * 1.0 / chartotal.total AS prob,
        COUNT(DISTINCT splitlex.word) * 1.0 / casetotals.total AS coverage
    FROM splitlex
    INNER JOIN (
        SELECT nouncase, COUNT(DISTINCT word) AS total
        FROM lex
        GROUP BY nouncase
    ) AS casetotals
    ON casetotals.nouncase = splitlex.nouncase
    INNER JOIN (
        SELECT char, num, COUNT(DISTINCT word || '-----' || nouncase) AS total
        FROM splitlex
        GROUP BY char, num
    ) AS chartotal
    ON splitlex.num = chartotal.num AND splitlex.char = chartotal.char
    GROUP BY splitlex.char, splitlex.nouncase, splitlex.num
    ) AS a
WHERE prob > 0.2
""")
c.fetchall()

[]

In [30]:
c.execute("""
CREATE INDEX IF NOT EXISTS nouncase_num_index
ON nouncase (num)
""")
conn.commit()

In [37]:
import time

def nouncase(word):
    start = time.time()
    c.execute("""
    SELECT s.word, s.char, nouncase.nouncase, MAX(nouncase.prob)
    FROM (
    WITH RECURSIVE
        cnt(x) AS (
            SELECT 1
            UNION ALL
            SELECT x+1 FROM cnt
            LIMIT 8
        )
        SELECT
            ? AS word,
            SUBSTR(?, LENGTH(?)-x+1, x) AS char,
            x AS num
        FROM cnt
    ) AS s
    INNER JOIN nouncase
    ON nouncase.char = s.char AND nouncase.num = s.num
    GROUP BY nouncase.nouncase
    ORDER BY prob DESC
    LIMIT 3
    """, (word, word, word))
    return c.fetchall(), time.time() - start

nouncase("ēdiens")

([('ēdiens', 'diens', 'nominatīvs', 1.0),
  ('ēdiens', 'ns', 'vokatīvs', 0.3011320754716981),
  ('ēdiens', 's', 'akuzatīvs', 0.2316308389874739)],
 0.3505682945251465)

In [38]:
nouncase("vakcīna")

([('vakcīna', 'īna', 'ģenitīvs', 0.5452322738386308),
  ('vakcīna', 'kcīna', 'vokatīvs', 0.5),
  ('vakcīna', 'kcīna', 'nominatīvs', 0.5)],
 0.3474299907684326)

In [39]:
nouncase("pilsoņi")

([('pilsoņi', 'oņi', 'nominatīvs', 0.5063291139240507),
  ('pilsoņi', 'ņi', 'vokatīvs', 0.5000452570601014),
  ('pilsoņi', 'i', 'datīvs', 0.38692873704781233)],
 0.3458237648010254)